In [24]:
import json
import os
from tqdm import tqdm
import shutil

In [13]:
## 변수 선언
root_dir="D:\\smartfarm_tomato\\leaves"
dst_dir="D:\\smartfarm_tomato\\data_for_yolo\\data"

cat=["train","valid"]
dtype=["images","labels"]

## annotation을 yolo coordinate로 
def convert_to_yolo_coords(xtl, ytl, xbr, ybr, img_width, img_height):
    """
    Convert bounding box coordinates from standard format to YOLO format.

    Standard format: {xtl, ytl, xbr, ybr} 
    YOLO format: {x_center, y_center, width, height}

    All YOLO coordinates are normalized between 0 and 1 relative to the image dimensions.

    Parameters:
    xtl (int or float): x-coordinate of the top-left corner
    ytl (int or float): y-coordinate of the top-left corner
    xbr (int or float): x-coordinate of the bottom-right corner
    ybr (int or float): y-coordinate of the bottom-right corner
    img_width (int): Width of the image
    img_height (int): Height of the image

    Returns:
    list of float: [x_center, y_center, width, height] in YOLO format
    """
    
    # Calculate the bounding box's center and width/height in pixel values
    x_center = (xtl + xbr) / 2
    y_center = (ytl + ybr) / 2
    width = xbr - xtl
    height = ybr - ytl

    # Normalize to image dimensions and return
    return [x_center / img_width, y_center / img_height, width / img_width, height / img_height]

In [36]:
## 문서접근
for index_1 in os.listdir(root_dir):
    for index_2 in tqdm(os.listdir(os.path.join(root_dir,index_1)),desc="Processing directories"):
        folder_path=os.path.join(root_dir,index_1,index_2)
        _, file_extension = os.path.splitext(folder_path)
        if file_extension != ".zip":
            for file_name in os.listdir(folder_path):
                dst_path=os.path.join(dst_dir,"temp",index_1,"labels")
                os.makedirs(dst_path, exist_ok=True)
                #json 파일 추출 후 저장 
                if file_name.endswith(".json"):
                    with open(os.path.join(folder_path,file_name),"r") as file:
                        data=json.load(file)
                        base_file_name = os.path.splitext(file_name)[0]
                        new_file_name = os.path.splitext(base_file_name)[0]+ '.txt'
                        with open(os.path.join(dst_path, new_file_name), 'w') as outfile:
                            #class id
                            if data["annotations"]["disease"] == 0:
                                class_id=0
                            else:
                                class_id=1
                            
                            #함수 연산                                                     
                            img_height, img_width = data["description"]["height"],data["description"]["width"]                            
                            point = data["annotations"]["points"][0]
                            xtl,ytl,xbr,ybr = point['xtl'],point['ytl'],point['xbr'],point['ybr']

                            # Convert to YOLO coordinates
                            x_center,y_center,width,height = convert_to_yolo_coords(xtl, ytl, xbr, ybr, img_width, img_height)                            
                            
                            # Write to output file
                            outfile.write(f'{class_id} {x_center} {y_center} {width} {height}\n')  
                
                # 사진 복사하기 
                elif file_name.endswith(".jpg"):
                    shutil.move(os.path.join(folder_path,file_name),os.path.join(dst_dir,index_1,"images",file_name))

Processing directories: 100%|██████████████████████████████████████████████████████████| 18/18 [01:42<00:00,  5.72s/it]


In [45]:
## 뭔가 오류로 인한 수동으로 split. 
import os
import shutil
from sklearn.model_selection import train_test_split

def move_files(file_list, src_img_dir, src_lbl_dir, dest_img_dir, dest_lbl_dir):
    for file_name in tqdm(file_list, desc='Moving files'):
        label_file_name = os.path.splitext(file_name)[0] + '.txt'
        try:
            shutil.move(os.path.join(src_img_dir, file_name), os.path.join(dest_img_dir, file_name))
            shutil.move(os.path.join(src_lbl_dir, label_file_name), os.path.join(dest_lbl_dir, label_file_name))
        except FileNotFoundError as e:
            print(f"File {file_name} or {label_file_name} not found. Skipping.")
            
# Source directories
src_img_dir = 'D:/smartfarm_tomato/data_for_yolo/data/temp/train/images'
src_lbl_dir = 'D:/smartfarm_tomato/data_for_yolo/data/temp/train/labels'

# Destination directories
train_img_dir = 'D:/smartfarm_tomato/data_for_yolo/data/inspection/train/images'
train_lbl_dir = 'D:/smartfarm_tomato/data_for_yolo/data/inspection/train/labels'
valid_img_dir = 'D:/smartfarm_tomato/data_for_yolo/data/inspection/valid/images'
valid_lbl_dir = 'D:/smartfarm_tomato/data_for_yolo/data/inspection/valid/labels'

# Ensure the destination directories exist
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(valid_img_dir, exist_ok=True)
os.makedirs(valid_lbl_dir, exist_ok=True)

# List to hold file names and labels
file_names = []
labels = []

# Iterate over all files in source directory
for file_name in os.listdir(src_img_dir):
    # Convert file_name to lower case before checking extension
    file_name_lower = file_name.lower()
    if file_name_lower.endswith('.jpg') or file_name_lower.endswith('.jpeg'):
        # Extract the "type" from the file name
        type_ = int(file_name.split('_')[2])
        
        file_names.append(file_name)
        labels.append(type_)

# Split file names into train and valid sets, stratified by "type"
train_files, valid_files, _, _ = train_test_split(file_names, labels, test_size=0.3, stratify=labels, random_state=42)

# Move files to new directories
move_files(train_files, src_img_dir, src_lbl_dir, train_img_dir, train_lbl_dir)
move_files(valid_files, src_img_dir, src_lbl_dir, valid_img_dir, valid_lbl_dir)

Moving files: 100%|████████████████████████████████████████████████████████████████| 5657/5657 [05:13<00:00, 18.06it/s]


In [23]:
sample="""{
    "description": {
        "image": "V006_77_0_00_11_01_13_0_c01_20201013_0000_S01.jpg",
        "date": "2020/10/13",
        "worker": "",
        "height": 3024,
        "width": 4032,
        "task": 77,
        "type": 0,
        "region": null
    },
    "annotations": {
        "disease": 0,
        "crop": 11,
        "area": 1,
        "grow": 13,
        "risk": 0,
        "points": [
            {
                "xtl": 1864,
                "ytl": 1516,
                "xbr": 2292,
                "ybr": 1880
            }
        ]
    }
}"""
study=json.loads(sample)
print(study.keys())
print(study["description"]["height"])
print(study["description"]["width"])
print(study["annotations"]["disease"])
print(study["annotations"]["points"])
print(study["annotations"]["disease"])

print(study["annotations"]["points"][0].values())

points = study["annotations"]["points"]

# Get the dimensions of the image
img_height, img_width = study["description"]["height"],study["description"]["width"]

point = study["annotations"]["points"][0]

xtl = point['xtl']
ytl = point['ytl']
xbr = point['xbr']
ybr = point['ybr']

    # Convert to YOLO coordinates
yolo_coords = convert_to_yolo_coords(xtl, ytl, xbr, ybr, img_width, img_height)
print(yolo_coords)

dict_keys(['description', 'annotations'])
3024
4032
0
[{'xtl': 1864, 'ytl': 1516, 'xbr': 2292, 'ybr': 1880}]
0
dict_values([1864, 1516, 2292, 1880])
[0.5153769841269841, 0.5615079365079365, 0.10615079365079365, 0.12037037037037036]


In [26]:
?shutil.move